In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from modules.config import PATH_SCENARIOS_REDUCED, PATH_SCENARIO_TREE_NODES

# Scenario Tree Structure Extraction
While generating the scenarios in 2.2 we ensured that the scenarios followed a tree structure, so that for example in the first time period all scenarios had the same demand values, while in the second period only half of all scenarios had the same values, etc. More precisely we ensured that the scenarios follow a tree structure where each node had exactly `N_REALIZATIONS` branches.  
The subset of the scenarios that we obtained in 2_3 still follows the tree structure, however not every node has to have the same number of branches. As a matter of fact we do not know the tree structure of the subset that we obtained. To formulate the stochastic program in our model, we need to know the tree structure to ensure non-anticipativity.  
Therefore we will examine the reduced scenarios to determine and save the tree structure.

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [5]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [6]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

In [7]:
node_df

node  prev_node
time     scenarios                 
00:00:00 0             0         -1
         1             0         -1
         2             0         -1
         3             0         -1
         4             0         -1
         5             0         -1
         6             0         -1
         7             0         -1
12:00:00 0             1          0
         1             2          0
         2             3          0
         3             4          0
         4             5          0
         5             6          0
         6             7          0
         7             8          0

In [8]:
os.makedirs(os.path.dirname(PATH_SCENARIO_TREE_NODES), exist_ok=True)
node_df.to_pickle(PATH_SCENARIO_TREE_NODES)